In [97]:
import tensorflow as tf
from utils.data import *
from utils.lstm_rnn import *
import os
import pandas as pd
if not os.path.isdir("save"):
    os.mkdir("save")
##### config #####
save_dataset = True # 儲存dataset，true: 生成dataset、false: 使用save/ 儲存的
window_size = 60 # (30,45,60)
shift = 15 # (5,15,30)
repeat = 100



##### config #####
dataPoint_per_signal = (180-window_size+1)//shift  * window_size
print("datapoint per signal:", dataPoint_per_signal)

datapoint per signal: 480


In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
for i in range(len(physical_devices)):
    tf.config.experimental.set_memory_growth(physical_devices[i], enable=True)

if save_dataset:
    X_raw, y = generate_time_series_dataset(window_size=window_size, shift=shift, repeat=repeat, dataPoint_per_signal=dataPoint_per_signal)
    #X = tf.reshape(X_raw, [y.shape[0],timestep,dataPoint_per_signal//timestep])
    np.save("save/X_raw-time.npy", X_raw)
    #np.save("save/X-time.npy", X.numpy())
    np.save("save/y-time.npy", y)
else:
    X_raw = np.load("save/X_raw-time.npy")
    #X = np.load("save/X-time.npy")
    y = np.load("save/y-time.npy")

#X_raw = tf.convert_to_tensor(X_raw)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
num of freq combination (160000, 2)


In [ ]:
"""
timestep=3

X = tf.reshape(X_raw, [y.shape[0],timestep,dataPoint_per_signal//timestep])
print(X.shape)
train_dataset, val_dataset, test_dataset = split_dataset(X, y)
X.shape[2]
"""

In [ ]:
try_timesteps=[15]#list(range(3,18,3))
try_units=[180,200]#[50,100,200,300,180]#list(range(10,200,20))
try_epochs=[100]
 
"""
try_timesteps=[3]
try_units=[180]#list(range(10,200,20))
try_epochs=[150]
"""
df_all = pd.DataFrame([], columns=[["loss", "out_1_loss", "out_2_loss", "out_1_acc", "out_2_acc", "timesteps", "units", "epoch"]])
for timesteps in try_timesteps:
    for units in try_units:
        for epoch in try_epochs:
            #X = tf.reshape(X_raw, [y.shape[0],timesteps,dataPoint_per_signal//timesteps])
            X = np.reshape(X_raw, (y.shape[0],timesteps,dataPoint_per_signal//timesteps))
            X = tf.convert_to_tensor(X)
            print(X.shape)
            train_dataset, val_dataset, test_dataset = split_dataset(X, y)
            
            model = construct_model(timesteps=timesteps, data_dim = X.shape[2], units=units)
            model, history = compile_fit_model(model,epochs=epoch,train_dataset=train_dataset,val_dataset=val_dataset)
            model.save("saved_model/(timestep)lstm-rnn-unit("+str(units)+")-timesteps("+str(timesteps)+")-epoch("+str(epoch)+")")
            results = model.evaluate(test_dataset)
            results.extend([timesteps, units, epoch])
            df = pd.DataFrame([results], columns=[["loss", "out_1_loss", "out_2_loss", "out_1_acc", "out_2_acc", "timesteps", "units", "epoch"]])
            df_all = df_all.append(df)
df_all.to_csv("save/result-time-w("+str(window_size)+")-sh("+str(shift)+").csv")

In [ ]:
df_all

In [ ]:
#testX_raw, y = generate_time_series_dataset(window_size=window_size, shift=shift, repeat=repeat)
#print( X_raw.shape, y.shape )

In [ ]:
X.shape

In [ ]:
window_size

#test function

In [ ]:
# SHIFT的方法 using tensorflow
range_ds = tf.data.Dataset.range(6)
window_size = 3
shift = 1
windows = range_ds.window(window_size, shift=shift)
def sub_to_batch(sub):
    return sub.batch(window_size, drop_remainder=True)

i=0
for example in windows.flat_map(sub_to_batch):
    print(example.numpy())
    i+=1
print("shape", i)

In [ ]:
data = np.arange(180)
def sliding_window(data=None, window_size = 6, shift = 1):
    return np.array(list( list(data[(x*shift):(x*shift)+window_size]) for x in range((data.shape[0]-window_size+1)//shift) ))

a = sliding_window(data, window_size = 60, shift = 10)
a

In [ ]:
a.flatten().shape

In [ ]:
a.reshape(-1)

In [ ]:
s = np_generate_signal([0,0],30)
print(s.shape)
s_window = sliding_window(data=s, window_size = 60, shift = 10).flatten()
s_window.shape

In [ ]:
window_size = 60 
shift = 1
_from=0
_to=20
sampling_rate=30
repeat=100
d=0.5

f1 = np.arange(_from,_to*2, d*2)
f2 = np.arange(_from, _to*2, d*2)

mesh = np.array(np.meshgrid(f1,f2))
# create labels 
combination = mesh.T.reshape(-1,2)
combination = np.sort(combination)
#combination = np.unique(combination, axis=0)
combination = np.repeat(combination, repeat, axis=0)
print("num of freq combination", combination.shape)
combination_onehot = tf.keras.utils.to_categorical(combination, num_classes=40)




i=0
data = np.zeros(combination.shape[0]*dataPoint_per_signal)

for f1,f2 in combination:
    start = time.time()
    s = np_generate_signal([f1,f2],sampling_rate)
    s_window = sliding_window(data=s, window_size = window_size, shift = shift).flatten()
    #data = tf.concat((data, s_window), axis=0)
    _start = time.time()
    #data = np.append(data, s_window, axis=0)
    data[dataPoint_per_signal*i:dataPoint_per_signal*(i+1)] =  s_window
    end = time.time()

    remain = combination.shape[0]-i
    print("["+str(remain)+"]|t:"+str(round((end-start)*remain,2)), end-start,end="\r")
    i+=1

#data = data[dataPoint_per_signal:]
print(data.shape)

In [ ]:
import matplotlib.pyplot as plt
c=159999
plt.plot(data[dataPoint_per_signal*c:dataPoint_per_signal*(c+1)])

In [ ]:
print(data.shape, s_window.shape, s.shape)

In [ ]:
data2 = np.append(data, s_window, axis=0)
data2

In [ ]:
a = np.zeros(combination.shape[0]*dataPoint_per_signal)
print(a.shape)
a


In [ ]:
a[0:2]=[1,2]
a

In [ ]:
a = np.arange(180)
b = sliding_window(data=a, window_size = 60, shift = 1)
print(b.shape, b.flatten().shape)
print(b)